Setup & install library – kalau kamu pakai environment baru, install dulu CatBoost / LightGBM dan imbalanced-learn.

In [10]:
!pip install pandas numpy scikit-learn catboost lightgbm imbalanced-learn


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Import library inti & set display options

In [11]:
import pandas as pd
import numpy as np
from pathlib import Path

# modelling
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

pd.set_option("display.max_columns", None)
DATA_DIR = Path("F:\lomba\intelectra\dataset")        # ganti jika file ada di folder lain


Load seluruh CSV

In [12]:
df_trans   = pd.read_csv(DATA_DIR / "train_transaction_data.csv")
df_trans_t = pd.read_csv(DATA_DIR / "test_transaction_data.csv")
df_member  = pd.read_csv(DATA_DIR / "member_data.csv")
df_prod    = pd.read_csv(DATA_DIR / "product_data.csv")
df_prog    = pd.read_csv(DATA_DIR / "program_data.csv")
df_label   = pd.read_csv(DATA_DIR / "train_label_data.csv")
print(df_trans.shape, df_member.shape, df_prod.shape, df_prog.shape, df_label.shape)


(130854, 8) (46401, 7) (27, 4) (27, 4) (40020, 2)


Quick EDA – cek proporsi label & preview data

In [13]:
print("Distribusi next_buy:")
print(df_label['next_buy'].value_counts(normalize=True))
df_trans.head()

Distribusi next_buy:
next_buy
0    0.93908
1    0.06092
Name: proportion, dtype: float64


,TransactionID,MemberID,Source,FK_PRODUCT_ID,FK_PROD_GRAM_ID,Qty,PricePerUnit,TransactionDatetime
0,ffd6c035bef03a2c386caf2552148fb2402b0b24,7ef72aa51aecb701dc5c4074480fcdf6,ELANG,27,52,1,NaN,2020-07-01 07:41:00 UTC
1,4fadf10aff7e250cc8d63badeaefdf383c0e1c13,577f1b9a093c2cec6398b1118f5d99ab,ALFAMART,22,39,1,290000.0,2020-07-01 07:24:00 UTC
2,7299df1aaf47b52017f58dd8e59df1fe37072670,e2ee74f248a74ed886a22f14348fbafd,EKTONG,22,39,1,263500.0,2020-07-01 09:13:00 UTC
3,22dae0e1e8bb87601eabd712721bd249739bc620,cefa8ef7469a8b4e6df3f745d4905000,CITRA,27,52,3,145000.0,2020-07-01 08:28:00 UTC
4,6a744d5d79e09b0c1646e604db1c4ec65eff73d7,5eecda17ddf06ed9d79f298b13f84785,INDOMARET,27,52,1,165900.0,2020-07-01 07:02:00 UTC


Join relasional ke tabel training

train_transaction ↔ member (MemberID)

product (FK_PRODUCT_ID)

program (FK_PROD_GRAM_ID)

merge dengan train_label

In [14]:
train = (df_trans
         .merge(df_member,  on='MemberID',      how='left')
         .merge(df_prod,    left_on='FK_PRODUCT_ID',   right_on='productID',  how='left')
         .merge(df_prog,    left_on='FK_PROD_GRAM_ID', right_on='programID',  how='left')
         .merge(df_label,   on='MemberID',      how='left')
        )

test  = (df_trans_t
         .merge(df_member,  on='MemberID',      how='left')
         .merge(df_prod,    left_on='FK_PRODUCT_ID',   right_on='productID',  how='left')
         .merge(df_prog,    left_on='FK_PROD_GRAM_ID', right_on='programID',  how='left')
        )

print(train.shape, test.shape)


KeyError: 'programID'

Feature Engineering ringkas

TotalSpending = Qty * PricePerUnit

datetime → Month, DayOfWeek, Recency

MemberAge, MemberTenure

drop kolom ID yang tak dipakai

In [ ]:
def create_features(df, is_train=True):
    df = df.copy()
    # datetime
    df['TransactionDatetime'] = pd.to_datetime(df['TransactionDatetime'])
    df['Month']       = df['TransactionDatetime'].dt.month
    df['DayOfWeek']   = df['TransactionDatetime'].dt.dayofweek
    
    # recency per Member
    df.sort_values(['MemberID', 'TransactionDatetime'], inplace=True)
    df['Recency'] = df.groupby('MemberID')['TransactionDatetime'].diff().dt.days
    df['Recency'].fillna(df['Recency'].median(), inplace=True)
    
    # numeric conversions
    df['Qty'] = pd.to_numeric(df['Qty'], errors='coerce')
    df['TotalSpending'] = df['Qty'] * df['PricePerUnit']
    
    # member age & tenure
    dob = pd.to_datetime(df['DateOfBirth'], errors='coerce')
    df['MemberAge'] = (pd.Timestamp('today') - dob).dt.days // 365
    joind = pd.to_datetime(df['JoinDate'], errors='coerce')
    df['MemberTenure'] = (pd.Timestamp('today') - joind).dt.days
    
    # pilih fitur
    keep_cols = ['MemberID','ProductCategory','ProductLevel','Source',
                 'Qty','TotalSpending','Month','DayOfWeek','Recency',
                 'MemberAge','MemberTenure']
    if is_train:
        keep_cols.append('next_buy')
    return df[keep_cols]

train_feat = create_features(train)
test_feat  = create_features(test, is_train=False)


Pre-processing – pisah X/y, handle imbalance (SMOTE), list kolom kategorikal

In [ ]:
y = train_feat['next_buy']
X = train_feat.drop(columns=['next_buy'])

cat_cols = ['ProductCategory','ProductLevel','Source']
for col in cat_cols:
    X[col] = X[col].astype('category')
    test_feat[col] = test_feat[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# Optional SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


Model : CatBoost (karena handle kategori otomatis) + evaluasi Balanced Accuracy

In [ ]:
model = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss',
    eval_metric='BalancedAccuracy',
    random_state=42,
    verbose=False
)

model.fit(
    X_train_res, y_train_res,
    eval_set=(X_val, y_val),
    cat_features=[X.columns.get_loc(c) for c in cat_cols],
    verbose=100
)

pred_val = model.predict(X_val)
print("Balanced Accuracy:", balanced_accuracy_score(y_val, pred_val))
print(classification_report(y_val, pred_val))


Cross-validation 5-fold (optional, tapi disarankan)

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(
    model, X, y, cv=cv, scoring='balanced_accuracy', n_jobs=-1)
print("Balanced Acc 5-fold:", scores.mean(), "±", scores.std())


Fit ulang di seluruh data training + prediksi data test

In [ ]:
model.fit(
    X, y,
    cat_features=[X.columns.get_loc(c) for c in cat_cols],
    verbose=False
)
test_pred = model.predict(test_feat)


Buat submission.csv – pastikan urutan MemberID sama dengan file sample

In [ ]:
submission = pd.DataFrame({
    'MemberID': test_feat['MemberID'],
    'next_buy': test_pred.astype(int)
})
submission.to_csv('submission.csv', index=False)
submission.head()
